# transformers库中 Pipeline的使用demo   

可以使用 `inspect` 来查看 `pipeline` 的源码来获取该函数输入参数要求

In [ ]:
from transformers import pipeline, pipelines

## 查看pipeline支持的任务类型   

In [ ]:
from transformers.pipelines import SUPPORTED_TASKS
print(SUPPORTED_TASKS.items())

### 上面的结果是非常乱的（kv形式数据），我们使用for循环来遍历出清晰客观的形式  

In [ ]:
for k, v in SUPPORTED_TASKS.items():
    print(k, "===>", v)
    print("-"*100)

### 根据不同的任务类型创建pipeline，需要注意的是❗️ 支持的任务和填写的参数 task是不一样的

在 `pipeline` 源码中，可以看到第一个参数 `task` 填写应该如下:

task (`str`):
            The task defining which pipeline will be returned. Currently accepted tasks are:

            - `"audio-classification"`: will return a [`AudioClassificationPipeline`].
            - `"automatic-speech-recognition"`: will return a [`AutomaticSpeechRecognitionPipeline`].
            - `"conversational"`: will return a [`ConversationalPipeline`].
            - `"depth-estimation"`: will return a [`DepthEstimationPipeline`].
            - `"document-question-answering"`: will return a [`DocumentQuestionAnsweringPipeline`].
            - `"feature-extraction"`: will return a [`FeatureExtractionPipeline`].
            - `"fill-mask"`: will return a [`FillMaskPipeline`]:.
            - `"image-classification"`: will return a [`ImageClassificationPipeline`].
            - `"image-feature-extraction"`: will return an [`ImageFeatureExtractionPipeline`].
            - `"image-segmentation"`: will return a [`ImageSegmentationPipeline`].
            - `"image-to-image"`: will return a [`ImageToImagePipeline`].
            - `"image-to-text"`: will return a [`ImageToTextPipeline`].
            - `"mask-generation"`: will return a [`MaskGenerationPipeline`].
            - `"object-detection"`: will return a [`ObjectDetectionPipeline`].
            - `"question-answering"`: will return a [`QuestionAnsweringPipeline`].
            - `"summarization"`: will return a [`SummarizationPipeline`].
            - `"table-question-answering"`: will return a [`TableQuestionAnsweringPipeline`].
            - `"text2text-generation"`: will return a [`Text2TextGenerationPipeline`].
            - `"text-classification"` (alias `"sentiment-analysis"` available): will return a
              [`TextClassificationPipeline`].
            - `"text-generation"`: will return a [`TextGenerationPipeline`]:.
            - `"text-to-audio"` (alias `"text-to-speech"` available): will return a [`TextToAudioPipeline`]:.
            - `"token-classification"` (alias `"ner"` available): will return a [`TokenClassificationPipeline`].
            - `"translation"`: will return a [`TranslationPipeline`].
            - `"translation_xx_to_yy"`: will return a [`TranslationPipeline`].
            - `"video-classification"`: will return a [`VideoClassificationPipeline`].
            - `"visual-question-answering"`: will return a [`VisualQuestionAnsweringPipeline`].
            - `"zero-shot-classification"`: will return a [`ZeroShotClassificationPipeline`].
            - `"zero-shot-image-classification"`: will return a [`ZeroShotImageClassificationPipeline`].
            - `"zero-shot-audio-classification"`: will return a [`ZeroShotAudioClassificationPipeline`].
            - `"zero-shot-object-detection"`: will return a [`ZeroShotObjectDetectionPipeline`].

### 比如根据pipeline创建 summarization 任务 （默认都是英文～用中文答案也是英文）     




可以看到，如果没有指定 model 直接执行 pipline 会出现一些警告      

In [20]:
pipe = pipeline('summarization') 

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [21]:
text = """I was born in the country, my family lived there before I was 6 years old. Then my parents moved to the city for their work, I had to stay away from my hometown. Though living in the city brings me a lot of convenience, and I eat the good food, I still miss country life all the time. I love to live in the country.

Living in the country, the time seems to be very slow. I woke up early in the morning and then took the walk. After eating the breakfast, it was about 8 o’clock. I went out to play with my friends or went to help my grandparents with their work. After doing these, it was just 11 o’clock. But in the city, I woke up at 9 o’clock, and then I ate the breakfast, the rest of the day was to play computer. How time flies to me.

I get so close to the nature and I find so much fun in the country life. I always want to live in my hometown. When I have the time, I will go back there and enjoy the moment.
"""

In [22]:
rs = pipe(text)
print(rs)

[{'summary_text': ' I was born in the country, my family lived there before I was 6 years old . My parents moved to the city for their work, I had to stay away from my hometown . Though living in the city brings me a lot of convenience, I still miss country life all the time .'}]


## 指定 model 和 task 创建 pipeline  

In [23]:
pipe = pipeline("summarization", model="Falconsai/text_summarization")

In [24]:
print(pipe(text))  

[{'summary_text': 'I was born in the country, my family lived there before I was 6 years old . My parents moved to the city for their work, I had to stay away from my hometown . Living in the city brings me a lot of convenience, and I eat the good food .'}]


## 使用pre-trained model 创建pipeline  

In [25]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
model = AutoModelForSeq2SeqLM.from_pretrained('Falconsai/text_summarization')
tokenizer = AutoTokenizer.from_pretrained('Falconsai/text_summarization')
pipe = pipeline('summarization', model=model, tokenizer=tokenizer)

In [26]:
print(pipe(text))

[{'summary_text': 'I was born in the country, my family lived there before I was 6 years old . My parents moved to the city for their work, I had to stay away from my hometown . Living in the city brings me a lot of convenience, and I eat the good food .'}]


In [27]:
pipe.model.device

device(type='cpu')

In [28]:
import torch
import time
times = []
for i in range(10):
    torch.cuda.synchronize()
    start = time.time()
    pipe(text)
    torch.cuda.synchronize()
    end = time.time()
    times.append(end - start)
print(sum(times) / 10)

1.8072702646255494


In [31]:
import torch

# 同步，确保所有之前的CUDA操作都已完成
torch.cuda.synchronize()
# 开始时间
start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)
start.record()

# 执行一些CUDA操作
pipe(text)

# 结束时间
end.record()

# 同步，确保所有CUDA操作都已完成
torch.cuda.synchronize()

# 计算运行时间
elapsed_time_ms = start.elapsed_time(end)
print(f'操作耗时: {elapsed_time_ms} 毫秒')


操作耗时: 1845.3974609375 毫秒


## 上面的是使用cpu推理，下面使用gpu推理看看时间

In [32]:
pipe = pipeline('summarization', model=model, tokenizer=tokenizer, device=0)
pipe.model.device

device(type='cuda', index=0)

In [33]:
import torch
import time
times = []
for i in range(10):
    torch.cuda.synchronize()
    start = time.time()
    pipe(text)
    torch.cuda.synchronize()
    end = time.time()
    times.append(end - start)
print(sum(times) / 10)

0.6993857622146606


In [34]:
import torch

# 同步，确保所有之前的CUDA操作都已完成
torch.cuda.synchronize()
# 开始时间
start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)
start.record()

# 执行一些CUDA操作
pipe(text)

# 结束时间
end.record()

# 同步，确保所有CUDA操作都已完成
torch.cuda.synchronize()

# 计算运行时间
elapsed_time_ms = start.elapsed_time(end)
print(f'操作耗时: {elapsed_time_ms} 毫秒')

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


操作耗时: 1003.6449584960938 毫秒


## 试一下问答任务  

In [35]:
qa = pipeline('question-answering', model="uer/roberta-base-chinese-extractive-qa")

config.json:   0%|          | 0.00/452 [00:00<?, ?B/s]

E:\software\anaconda3\envs\transformers\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\yuehu\.cache\huggingface\hub\models--uer--roberta-base-chinese-extractive-qa. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


pytorch_model.bin:   0%|          | 0.00/407M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/216 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

### 使用说明 
1. `question`参数：这个参数用于提供用户想要提问的问题。在这个例子中，问题是“广东省省会是哪里？”。这个参数告诉模型需要寻找答案的问题内容。

2. `context`参数：这个参数提供了一个文本段落，其中可能包含问题的答案。在这个例子中，上下文是“广东省省会是广州”。模型会在这个上下文中搜索并提取出问题的答案。

`max_answer_len`参数是可选的，它指定了模型生成的答案的最大长度。在这个例子中，设置为2，意味着模型给出的答案不会超过2个字符。




In [39]:
qa(question="广东省省会是哪里？", context="广东省省会是广州", max_answer_len=2)

{'score': 0.8817171454429626, 'start': 6, 'end': 8, 'answer': '广州'}

# pipeline实现原理和详细步骤
Hugging Face Transformers库中的`pipeline`是一个高级API，它简化了常见NLP任务的实现过程。当你使用`pipeline`时，它在背后为你处理了很多细节，比如加载模型、处理输入数据、调用模型以及后处理输出结果。

下面是`pipeline`背后的一般步骤和原理：
1. **加载模型和 tokenizer**：
   - `pipeline`首先根据你提供的任务名称（如`'question-answering'`）和模型名称（如`'uer/roberta-base-chinese-extractive-qa'`）加载相应的预训练模型和分词器（tokenizer）。
   - 分词器用于将输入文本（问题和上下文）转换为模型可以理解的数字表示（即词嵌入）。
2. **预处理输入数据**：
   - `pipeline`使用分词器对输入的问题和上下文进行预处理，包括分词、转换为词嵌入、添加必要的特殊标记（如BERT的`[CLS]`和`[SEP]`）等。
   - 预处理后的数据会被组织成模型期望的格式。
3. **调用模型**：
   - 预处理后的数据被送入加载的模型中。
   - 模型根据输入数据计算输出，对于问答任务，这通常是一个表示答案开始和结束位置的分数分布。
4. **后处理输出**：
   - `pipeline`根据模型的输出进行后处理，例如，对于问答任务，它会从分数分布中找出最可能的答案开始和结束位置。
   - 然后，它会从原始上下文中提取出对应的文本作为答案。
5. **返回结果**：  
   - 最后，`pipeline`返回处理后的结果，通常是问题的答案。

在之前问答任务的上下文中，`pipeline`使用了一个名为“uer/roberta-base-chinese-extractive-qa”的模型，这是一个基于RoBERTa的模型，专门用于中文文本的提取式问答。这意味着它不是生成答案，而是从给定的上下文中提取答案。

**`pipeline`的设计目的是为了简化常见NLP任务的实现，让用户可以不必深入了解模型和数据处理的具体细节，就能快速实现和部署模型。**


In [41]:
from transformers import *
import torch

E:\software\anaconda3\envs\transformers\Lib\site-packages\transformers\deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(


## step1 加载模型和tokenizer

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

tokenizer = AutoTokenizer.from_pretrained("uer/roberta-base-chinese-extractive-qa")
model = AutoModelForQuestionAnswering.from_pretrained("uer/roberta-base-chinese-extractive-qa")

## step2 预处理输入数据         
貌似不加 `add_specical_tokens=True` 也没什么问题

In [78]:
question, context = "广东省省会是哪里？", "广东省省会是广州"
# inputs = tokenizer(question, context, return_tensors="pt", add_special_tokens=True)
inputs = tokenizer(question, context, return_tensors="pt")
inputs

{'input_ids': tensor([[ 101, 2408,  691, 4689, 4689,  833, 3221, 1525, 7027, 8043,  102, 2408,
          691, 4689, 4689,  833, 3221, 2408, 2336,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

## step3 调用模型

In [79]:
with torch.no_grad():
    outputs = model(**inputs)
outputs

QuestionAnsweringModelOutput(loss=None, start_logits=tensor([[ 2.8356, -2.5820, -6.5702, -8.1818, -8.0138, -8.1900, -6.5748, -4.0969,
         -7.1589, -8.1996, -7.6051, -1.9516, -6.4490, -6.2248, -4.5364, -6.3773,
         -4.8039,  5.6266, -0.9436, -7.5889]]), end_logits=tensor([[ 2.8049, -3.7779, -0.8281, -3.6546, -5.3831, -4.6677, -6.7706, -5.0973,
         -3.9445, -8.3965, -8.5465, -5.2114, -2.4174, -3.6956, -5.1664, -4.2346,
         -6.8528, -1.7553,  5.5331, -8.3205]]), hidden_states=None, attentions=None)

## step4 后处理输出  

In [80]:
# start_logits = outputs.start_logits
# end_logits = outputs.end_logits
# answer_start = torch.argmax(start_logits)
# answer_end = torch.argmax(end_logits)
# answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end+1]))
# answer

start_logits = outputs.start_logits
end_logits = outputs.end_logits
answer_start = torch.argmax(start_logits)
answer_end = torch.argmax(end_logits)

# 确保结束位置在开始位置之后
if answer_end < answer_start:
    answer_end = answer_start + 2  # 至少包含一个字符 

## step5 返回结果  

In [81]:
answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end+1]))
print(answer)

广 州
